In [ ]:
YMPy_path = 'C:/Users/elgui/Documents/Emr Lab Post Doc/Bioinformatics and Modelling/Python/EMRyeast/YeastMorphologyPypeline/'

import sys
sys.path
sys.path.append(YMPy_path)

In [ ]:
import YMPy
import pickle
import numpy as np
import datetime
import os

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
targetFolder = 'C:/Users/elgui/Documents/Emr Lab Post Doc/microscopy/Art1Quant'

In [ ]:
# local function parameters
rolloff = 64
nChannels = 3
zFirst = True
bwChannel = 2
bfAnomalyShiftVector = [2,-1]
minAngle = 22
minLength = 5
closeRadius = 3
minBudSize = 75
cortexWidth = 10
bufferSize = 5
showProgress = True
mkrChannel = 0
borderSize = 10
golgiRadius = 7
rgbScalingFactors = [0.05, 0.1]
gryScalingFactors = [0.2, 0.2]

In [ ]:
print('beginning analysis of \n',targetFolder,'\n at ', datetime.datetime.now())
# initialize experiment variables
totalResults = []
totalQC = []
fieldsAnalyzed = []
totalMcl = []

resultsDirectory = targetFolder + '/results/'
if not os.path.exists(resultsDirectory):
    os.makedirs(resultsDirectory)

folderData = YMPy.batchParse(targetFolder, expIDloc=[0,6])
nFields = folderData['nFields']
imageNameList = folderData['imagenameList']
pathList = folderData['pathlist']
expIDlist = folderData['expIDlist']

globalExtremaG = YMPy.batchIntensityScale(
        folderData, channel=1, showProgress=True)
globalExtremaR = YMPy.batchIntensityScale(
        folderData, channel=0, showProgress=True)
globalMinG = globalExtremaG['globalmin']
globalMaxG = globalExtremaG['globalmax']
globalMinR = globalExtremaR['globalmin']
globalMaxR = globalExtremaR['globalmax']

for field in range(nFields):
    print('starting image: ', imageNameList[field])
    # read image
    dvImage = YMPy.basicDVreader(pathList[field], rolloff, nChannels,
                                       zFirst)
    # find cells from brightfield step 1
    bwCellZstack = YMPy.makeCellzStack(dvImage, bwChannel, showProgress)
    # find cells from brightfield step 2
    nZslices = dvImage.shape[1]        
    for z in range(nZslices):
        bwCellZstack[z,:,:] = YMPy.helpers.correctBFanomaly(
            bwCellZstack[z,:,:], bfAnomalyShiftVector)
    # find cells from brightfield step 3
    rawMcl = YMPy.cellsFromZstack(bwCellZstack, showProgress)[0]
    # find cells from brightfield step 4
    unbufferedMcl = YMPy.bfCellMorphCleanup(rawMcl, showProgress,
                                                  minAngle, minLength,
                                                  closeRadius, minBudSize)
    # unbufferedMcl is the best guess at the 'true outside edge' of the cells;
    # use it as the starting point to find a 10pixel thick cortex
    cortexMcl = YMPy.labelCortex_mcl(unbufferedMcl, cortexWidth)
    # because the bright field and fluorescence are not perfectly aligned, and
    # to handle inaccuracies in edge finding, also buffer out from the outside
    # edge
    buffer = YMPy.buffer_mcl(unbufferedMcl, bufferSize, showProgress)
    # merge this buffer onto the unbufferedMcl and the cortexMcl
    masterCellLabel = YMPy.merge_labelMcl(unbufferedMcl, buffer)
    cortexMclBuffered = YMPy.merge_labelMcl(cortexMcl, buffer)
    # use Otsu thresholding on the max projection of mCh-Sec7 to find golgi
    golgiMcl = YMPy.labelMaxproj(masterCellLabel, dvImage, mkrChannel)
    golgiCirclesMcl = YMPy.centroidCirclesMcl(
            golgiMcl.astype('bool'), masterCellLabel,
            golgiRadius, iterations=5)
    # subtract so that golgi localization has precedence over cortical
    # localization
    cortexMinusGolgi = YMPy.subtract_labelMcl(cortexMclBuffered,golgiMcl)
    ctxMinusGolgiCC = YMPy.subtract_labelMcl(cortexMclBuffered,
                                                   golgiCirclesMcl)
    # prepare for measuring:
    # measure Art1-mNG in the middle z-slice
    primaryImage = {'Art1-mNG':dvImage[1,3,:,:]}
    # measure against buffered cortex, golgi, and non-golgi buffered cortex
    refMclDict = {'cortex(buffered)':cortexMclBuffered,
                  'golgi':golgiMcl,
                  'nonGolgicortex(buffered)':cortexMinusGolgi,
                  'golgiCircles':golgiCirclesMcl,
                  'nonGolgiCirclesCortext(buffered)':ctxMinusGolgiCC
                  }
    # also record field wide information
    expID = expIDlist[field]
    imageName = imageNameList[field]
    # measure
    results = YMPy.measure_cells(primaryImage, masterCellLabel,
                                 refMclDict, imageName, expID, field,
                                 globalMinG, globalMaxG, showProgress)
    # add measurements from each field to total results
    totalResults = np.concatenate((totalResults,results))
    # quality control prep
    print('preparing quality control information')
    greenFluor = dvImage[1,3,:,:].astype(float)
    greenFluorScaled = ((greenFluor.astype('float')-globalMinG)
                        /(globalMaxG-globalMinG))
    redFluor = np.amax(dvImage[0,:,:,:],axis=0).astype(float)
    redFluorScaled = ((redFluor.astype('float')-globalMinR)
                        /(globalMaxR-globalMinR))
    qcMclList = [ctxMinusGolgiCC,golgiCirclesMcl]
    rgbQC = YMPy.prep_rgbQCImage(greenFluorScaled, redFluorScaled,
                                       qcMclList, rgbScalingFactors)

    # add qcStack to totalQC
    totalQC.append(rgbQC)
    # record field as analyzed
    fieldsAnalyzed.append(field)
    # save masterCellLabel
    totalMcl.append(masterCellLabel)
    # pool and save
    print('saving progress')
    
    pickle.dump({'totalResults':totalResults,
               'totalQC':totalQC,
               'fieldsAnalyzed':fieldsAnalyzed,
               'totalMcl':totalMcl
               }, open(targetFolder 
                       + '/results/'
                       + str(datetime.datetime.now().date()) 
                       + '_analysis.p', 'wb'))
    print(imageNameList[field],' complete at ',datetime.datetime.now())

In [ ]:
resultsDirectory = targetFolder + '/results/'
resultsName = '/2018-06-20_analysis.p'
qcAutosaveName = '/qcAutosave.p'
resultsDataPath = resultsDirectory + resultsName
qcAutosavePath = resultsDirectory + qcAutosaveName

expIDcategory = 'AA'

resultsData = YMPy.qcDisplay.initializeQC(resultsDataPath, expIDcategory, '20180621')

In [ ]:
folderData = YMPy.batchParse(targetFolder, expIDloc=[0,6])
pathList = folderData['pathlist']

b1,b2,b3,b4 = YMPy.qcDisplay.makeQCbuttons()
global status_list
randLookupStart=0
objs = YMPy.qcDisplay.makeQC_clickfunctions(
    randLookupStart, resultsData, resultsData['totalResults'], pathList, ['mCh-Sec7','Art1-mNG'], resultsDirectory, autosave=True) 
click_b1, click_b2, click_b3, click_b4, out1, out2, status_list = objs
    
b1.on_click(click_b1)
b2.on_click(click_b2)
b3.on_click(click_b3)
b4.on_click(click_b4)

plt.ioff()
qcDict = YMPy.qcDisplay.makeQCframe(randLookupStart, resultsData, resultsData['totalResults'], pathList)
with out1:
    YMPy.qcDisplay.frameDisplay(qcDict,['mCh-Sec7','Art1-mNG'])
widgets.HBox([out1, widgets.VBox([b1,b2,b3,b4,out2])])